In [24]:
import tensorflow as tf
from keras import models,layers
import matplotlib.pyplot as plt

In [25]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [26]:
gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)  # Set memory growth dynamically
#     except RuntimeError as e:
#         print(e)


In [27]:
# if gpus:
#     try:
#         tf.config.set_logical_device_configuration(
#             gpus[0],
#             [tf.config.LogicalDeviceConfiguration(memory_limit=512)]
#         )
#         logical_gpus = tf.config.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,",len(logical_gpus),"Logical GPUs")
#     except RuntimeError as e:
#         print(e)

In [28]:
IMAGE_SIZE = 256
BATCH_SIZE = 16
CHANNELS = 3
EPOCHS = 50

In [29]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "../PlantVillage/train",
    shuffle=True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE,

)

# dataset = dataset.cache().shuffle(100).prefetch(buffer_size=tf.data.AUTOTUNE)


Found 3600 files belonging to 3 classes.


In [30]:
class_names = dataset.class_names

In [31]:
class_names

['Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___healthy']

In [32]:
len(dataset)

225

In [33]:
# plt.figure(figsize=(20,20))
# for image_batch, label_batch in dataset.take(1):
#   for i in range(12):
#     ax = plt.subplot(3,4,i+1)
#     plt.imshow(image_batch[i].numpy().astype("uint8"))
#     plt.title(class_names[label_batch[i]])
#     plt.axis("off")
  # print(image_batch[0].numpy())
  # print(label_batch)

In [34]:
#80% training , 10% validation, 10% testing
# train_ds =  dataset.take(len(dataset)*0.8)
# val_and_train_ds = dataset.skip(len(dataset)*0.8)
# val_ds = val_and_train_ds.take(len(val_and_train_ds)*0.5)
# test_ds = val_and_train_ds.skip(len(val_and_train_ds)*0.5)


In [35]:
def get_dataset_partitions_tf(ds,train_split = 0.8, val_split = 0.1, test_split=0.1, shuffle=True, shuffle_size = 10000):
    ds_size = len(ds)

    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)

    train_size = int(ds_size * train_split)
    val_size = int( ds_size * val_split)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    return train_ds, val_ds, test_ds

In [36]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [37]:
len(train_ds)

180

In [38]:
len(val_ds)

22

In [39]:
len(test_ds)

23

In [40]:
# to load the next batch using cpu while the gpu does training
train_ds = train_ds.cache().shuffle(100).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(100).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(100).prefetch(buffer_size=tf.data.AUTOTUNE)

In [41]:
#The image given can be off different dimensions so we resize,
# the image has to be converted into grayscale so we rescale
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    layers.Rescaling(1.0/255)
])

In [42]:
# data augmentation is used, what is does is basically we have images, which we convert into it having horizontal flip, increased contrast, zoom and rotation
# Because we are not sure what device is used to capture the image in our app, it can have different processing done to give varying colors or orientation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])


In [43]:
input_shape = (BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,CHANNELS)
n_classes = 3

model = models.Sequential([
    resize_and_rescale,
    data_augmentation,
    
    layers.Conv2D(32,(3,3),activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    # layers.Conv2D(64,(3,3),activation='relu'),
    # layers.MaxPooling2D((2,2)),
    # layers.Conv2D(64,(3,3),activation='relu'),
    # layers.MaxPooling2D((2,2)),
    # layers.Conv2D(64,(3,3),activation='relu'),
    # layers.MaxPooling2D((2,2)),
    
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(n_classes,activation='softmax'),
])

model.build(input_shape=input_shape);

In [44]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (16, 256, 256, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (16, 256, 256, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (16, 254, 254, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (16, 127, 127, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (16, 125, 125, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (16, 62, 62, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (16, 60, 60, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (16, 30, 30, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (16, 57600)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (16, 64)               │     3,686,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (16, 3)                │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,742,979 (14.28 MB)

 Trainable params: 3,742,979 (14.28 MB)

 Non-trainable params: 0 (0.00 B)

In [45]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy'],
)

In [46]:

# history = model.fit(
#     train_ds,
#     epochs=EPOCHS,
#     batch_size=BATCH_SIZE,
#     verbose=1,
#     validation_data=val_ds,
# )


class ClearSessionCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        tf.keras.backend.clear_session()

# Fit the model with callbacks
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    verbose=1,
    callbacks=[ClearSessionCallback()]
)


Epoch 1/50
180/180 ━━━━━━━━━━━━━━━━━━━━ 13s 51ms/step - accuracy: 0.5072 - loss: 0.9889
Epoch 2/50
180/180 ━━━━━━━━━━━━━━━━━━━━ 8s 46ms/step - accuracy: 0.8286 - loss: 0.3768
Epoch 3/50
180/180 ━━━━━━━━━━━━━━━━━━━━ 8s 46ms/step - accuracy: 0.8548 - loss: 0.3160
Epoch 4/50
180/180 ━━━━━━━━━━━━━━━━━━━━ 8s 46ms/step - accuracy: 0.8661 - loss: 0.2957
Epoch 5/50
180/180 ━━━━━━━━━━━━━━━━━━━━ 8s 46ms/step - accuracy: 0.8707 - loss: 0.3025
Epoch 6/50
180/180 ━━━━━━━━━━━━━━━━━━━━ 8s 46ms/step - accuracy: 0.8906 - loss: 0.2830
Epoch 7/50
180/180 ━━━━━━━━━━━━━━━━━━━━ 8s 45ms/step - accuracy: 0.9066 - loss: 0.2290
Epoch 8/50
180/180 ━━━━━━━━━━━━━━━━━━━━ 8s 45ms/step - accuracy: 0.9236 - loss: 0.1903
Epoch 9/50
180/180 ━━━━━━━━━━━━━━━━━━━━ 8s 46ms/step - accuracy: 0.8992 - loss: 0.2559
Epoch 10/50
180/180 ━━━━━━━━━━━━━━━━━━━━ 9s 50ms/step - accuracy: 0.9403 - loss: 0.1617
Epoch 11/50
180/180 ━━━━━━━━━━━━━━━━━━━━ 9s 46ms/step - accuracy: 0.9293 - loss: 0.1896
Epoch 12/50
180/180 ━━━━━━━━━━━━━━━━━━━━

In [1]:
scores = model.evaluate(test_ds)

NameError: name 'model' is not defined